#  User-Friendly FFDI Analysis Notebook

This notebook contains simplified, beginner-friendly code blocks for performing analysis on Fire Forest Danger Index (FFDI) raster data.

Each section:
- Explains the purpose of the code
- Provides step-by-step process
- Uses meaningful variable names and plots

Please make sure to update any placeholder paths with your actual data directories before running.


In [ ]:
# SECTION 1: Reading and summarizing monthly FFDI from multiple methods

import os
import glob
import re
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# STEP 1: Define folders for each method you're comparing
folders = {
    "Method1": r'C:/.../FFDI_final',     # <- Update with your actual path
    "Method2": r'C:/.../FFDI_final2',    # <- Each method has different FFDI rasters
    "Method3": r'C:/.../FFDI_final3',
    "Method4": r'C:/.../FFDI_final4',
    "Method5": r'C:/.../FFDI_mean'
}

# STEP 2: Define pattern to extract YYYY-MM from filenames like "ffdi_2013-05.tif"
date_pattern = re.compile(r'\d{4}-\d{2}')
method_dfs = {}  # Dictionary to store time series data per method

# STEP 3: Loop through each folder and read TIFF files
for method, folder_path in folders.items():
    print(f"Processing method: {method}")
    tif_files = glob.glob(os.path.join(folder_path, "*.tif"))

    records = []  # Stores (date, FFDI value) for the method

    for file in tif_files:
        match = date_pattern.search(os.path.basename(file))
        if not match:
            continue
        date = pd.to_datetime(match.group() + "-01")

        with rasterio.open(file) as src:
            data = src.read(1)  # Read first band
            valid = data[~np.isnan(data)]  # Filter no-data pixels
            if valid.size > 0:
                records.append((date, np.mean(valid)))

    # Store in DataFrame
    if records:
        df = pd.DataFrame(records, columns=["Month", method])
        df.set_index("Month", inplace=True)
        method_dfs[method] = df

# STEP 4: Combine all methods into a single DataFrame by date
if method_dfs:
    combined_df = pd.concat(method_dfs.values(), axis=1)
    combined_df.sort_index(inplace=True)

    # STEP 5: Plot FFDI over time for each method
    plt.figure(figsize=(14, 6))
    for method in combined_df.columns:
        plt.plot(combined_df.index, combined_df[method], label=method)
    plt.title("Monthly Average FFDI Comparison")
    plt.xlabel("Month")
    plt.ylabel("FFDI")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # STEP 6: Print stats and correlation
    print("\nSummary statistics:")
    print(combined_df.describe())
    print("\nCorrelation between methods:")
    print(combined_df.corr())

else:
    print("No data was found or processed.")


## SECTION 2: KMeans Clustering on Raster Image Features


This section loads each TIFF file as an image, flattens it into a 1D vector, and performs KMeans clustering
to group similar patterns of FFDI values over time. It helps in identifying clusters of fire danger behavior.

Steps:
- Read TIFFs as grayscale images
- Resize and flatten
- Standardize
- Cluster using KMeans
- Use inertia and silhouette scores to determine optimal cluster count


In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# 1. Set the path to your input folder containing the TIFF files
input_folder = "C:/Users/gades/Desktop/Thesis/datasets/terra_var/FFDI_finaldf90"  # <-- Replace with your folder path
tif_files = glob.glob(os.path.join(input_folder, "*.tif"))

# 2. Feature extraction from each TIFF image:
# Here, we convert the image to grayscale, resize it to 64x64 pixels (for example),
# and then flatten it into a 1D feature vector.
features = []
for file in tif_files:
    try:
        img = Image.open(file).convert("L")  # Convert to grayscale
        img_resized = img.resize((64, 64))    # Resize to 64x64 pixels
        img_array = np.array(img_resized).flatten()  # Flatten to a 1D vector
        features.append(img_array)
    except Exception as e:
        print(f"Error processing {file}: {e}")

# Convert list of feature vectors to a NumPy array
data = np.array(features)
print(f"Extracted features shape: {data.shape}")

# 3. Impute any missing values (if any exist)
imputer = SimpleImputer(strategy="mean")
data_imputed = imputer.fit_transform(data)
print(f"NANs before imputation: {np.sum(np.isnan(data))}")
print(f"NANs after imputation: {np.sum(np.isnan(data_imputed))}")

# 4. Standardize the features
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_imputed)

# 5. Evaluate a range of k values for clustering using KMeans
inertias = []
silhouette_scores = []
k_range = range(2, 11)  # testing k from 2 to 10

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(data_scaled)
    
    # Record the inertia and silhouette score
    inertias.append(kmeans.inertia_)
    silhouette = silhouette_score(data_scaled, labels)
    silhouette_scores.append(silhouette)
    
    print(f"k = {k}: inertia = {kmeans.inertia_:.2f}, silhouette score = {silhouette:.3f}")

# 6. Plot the results for the Elbow Method and Silhouette Scores
plt.figure(figsize=(12, 5))

# Elbow Plot (Inertia vs. k)
plt.subplot(1, 2, 1)
plt.plot(list(k_range), inertias, marker='o')
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Inertia")
plt.title("Elbow Method")

# Silhouette Scores Plot
plt.subplot(1, 2, 2)
plt.plot(list(k_range), silhouette_scores, marker='o', color='red')
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Scores")

plt.tight_layout()
plt.show()

## SECTION 3: Outlier Detection Using IQR Method on Each Raster


This section identifies statistical outliers (extremely high or low FFDI values) in each TIFF file using the
Interquartile Range (IQR) method. Boxplots help visualize the outlier range.


In [ ]:
import os
import glob
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Define the folder containing your monthly TIFF files
folder_path = 'C:/Users/gades/Desktop/Thesis/datasets/terra_var/FFDI_finaldf90'  # Update this if needed

# List all .tif files in the folder
tif_files = glob.glob(os.path.join(folder_path, "*.tif"))
print(f"Found {len(tif_files)} TIFF files.\n")

# List for files with detected outliers
files_with_outliers = []

for file in tif_files:
    try:
        with rasterio.open(file) as src:
            print(f"Processing file: {file}")
            data = src.read(1)
            # Remove NaN values (since your no-data is NaN)
            data = data[~np.isnan(data)]
            if data.size == 0:
                print("  No valid data found in this file.\n")
                continue

            # Flatten and convert to a pandas Series
            ffdi_series = pd.Series(data.flatten())
            
            # ---------- Outlier Detection using the IQR Method ----------
            Q1 = ffdi_series.quantile(0.25)
            Q3 = ffdi_series.quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            # Identify outliers
            outliers = ffdi_series[(ffdi_series < lower_bound) | (ffdi_series > upper_bound)]
            num_outliers = outliers.size
            
            # Only record files that have outliers
            if num_outliers > 0:
                files_with_outliers.append((os.path.basename(file), num_outliers))
                print(f"  Number of outliers: {num_outliers}")
                print(f"  Outlier range: < {lower_bound:.2f} or > {upper_bound:.2f}\n")
                
                # ---------- Plot the Box Plot (Optional) ----------
                plt.figure(figsize=(8, 4))
                plt.boxplot(ffdi_series, vert=False, showfliers=True, whis=1.5)
                plt.title(f'Box Plot of FFDI Values\n{os.path.basename(file)}')
                plt.xlabel('FFDI')
                plt.xlim(0, 50)  # Force x-axis to the known FFDI range
                plt.show()
            else:
                print("  No outliers detected.\n")
            
    except Exception as e:
        print(f"Error processing {file}: {e}\n")

# Summary: Print only the files with detected outliers and their count
print("Files with detected outliers:")
for fname, count in files_with_outliers:
    print(f"  {fname}: {count} outliers")
print(f"\nTotal number of TIFF files with outliers: {len(files_with_outliers)}")

## SECTION 4: Spatiotemporal Trend Analysis with KMeans and Mann-Kendall Test


This advanced section clusters spatial pixels by mean FFDI, computes monthly time series per cluster, and then
performs Mann-Kendall trend test with Sen's slope to quantify trends over time.


In [ ]:
import os
import numpy as np
import rasterio
from rasterio.windows import from_bounds
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
import pandas as pd
import pymannkendall as mk

# ─── CONFIG ──────────────────────────────────────────────────────────────────
ffdi_folder = r"C:\Users\gades\Desktop\Thesis\datasets\terra_var\FFDI_finaldf90"
n_clusters  = 3


# ─── 1) LOAD FILE LIST & DATES ────────────────────────────────────────────────
files = sorted(f for f in os.listdir(ffdi_folder) if f.lower().endswith(".tif"))
paths = [os.path.join(ffdi_folder, f) for f in files]
dates = [pd.to_datetime(f.replace("ffdi_","").replace(".tif","")) for f in files]

# ─── 2) REFERENCE METADATA ─────────────────────────────────────────────────────
with rasterio.open(paths[0]) as src0:
    bounds = src0.bounds
    w, h   = src0.width, src0.height
    nodata = src0.nodata

# ─── 3) STACK PIXEL×TIME ───────────────────────────────────────────────────────
X = np.empty((w*h, len(paths)), dtype=np.float32)
for i, p in enumerate(paths):
    with rasterio.open(p) as src:
        window = from_bounds(*bounds, transform=src.transform)
        arr = src.read(1, window=window,
                       out_shape=(h, w),
                       resampling=rasterio.enums.Resampling.bilinear)
    X[:,i] = arr.flatten()

# ─── 4) IMPUTE & K-MEANS ON PIXEL MEANS ───────────────────────────────────────
X = SimpleImputer(strategy="mean").fit_transform(X)
pixel_means = X.mean(axis=1).reshape((h, w))a
labels = KMeans(n_clusters=n_clusters, random_state=0)\
            .fit_predict(pixel_means.flatten().reshape(-1,1))
cluster_map = labels.reshape((h, w))

# ─── 5) COMPUTE MONTHLY MIN/MEAN/MAX FOR EACH CLUSTER ─────────────────────────
cluster_ts = {}
for k in range(n_clusters):
    mask = (cluster_map == k)
    vals = X[mask.flatten(), :]
    cluster_ts[k] = {
        "min":  np.nanmin(vals, axis=0),
        "mean": np.nanmean(vals, axis=0),
        "max":  np.nanmax(vals, axis=0)
    }

# ─── 6) TREND ANALYSIS: SEN’S SLOPE + MANN–KENDALL ───────────────────────────
trends = {}
mk_results = {}
for k in range(n_clusters):
    series = cluster_ts[k]["mean"]
    sen = mk.sens_slope(series)
    mk_test = mk.original_test(series)
    trends[k] = sen.slope
    mk_results[k] = {
        "slope": sen.slope,
        "p_value": mk_test.p
    }

# ─── 7) PLOT: GREY ENVELOPE + MEANS + TREND LINES ────────────────────────────
fig, ax = plt.subplots(figsize=(12, 6))
colors = ['C0','C1','C2']

# draw grey envelope once
for k in range(n_clusters):
    ax.fill_between(dates,
                    cluster_ts[k]["min"],
                    cluster_ts[k]["max"],
                    color='lightgrey', alpha=0.3)

# plot each cluster
for k in range(n_clusters):
    ts = pd.Series(cluster_ts[k]["mean"], index=dates)
    ax.plot(dates, ts.values,
            color=colors[k], linewidth=2,
            label=f"Cluster {k+1} Mean")
    idx = np.arange(len(dates))
    trend_line = ts.values[0] + trends[k]*idx
    ax.plot(dates, trend_line,
            color=colors[k], linestyle='--', linewidth=1.5,
            label=f"Cluster {k+1} Trend (s={trends[k]:.3f}, p={mk_results[k]['p_value']:.3f})")

# formatting

ax.set_xlabel("Date")
ax.set_ylabel("FFDI Value")
ax.grid(True, linestyle=':', linewidth=0.5)
ax.legend(ncol=2, fontsize=9)

plt.tight_layout()
plt.savefig(out_png, dpi=300)
plt.show()

# ─── 8) PRINT MANN–KENDALL RESULTS ────────────────────────────────────────────
print("Mann–Kendall Trend Test Results per Cluster:")
for k in range(n_clusters):
    r = mk_results[k]
    direction = "increasing" if r["slope"]>0 else "decreasing"
    print(f" Cluster {k+1}: slope = {r['slope']:.3f}, p-value = {r['p_value']:.3f} ({direction})")

print(f"\n✅ Plot saved to: {out_png}")

## SECTION 5: Spatial Cluster Map of Temporal FFDI Time Series


Clusters each pixel based on its full time-series FFDI behavior. This visualizes spatial patterns of similar
temporal dynamics in FFDI.


In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# ------------------------------
# Load your FFDI Data (same as before)
# ------------------------------
# Set the input folder containing the GeoTIFF files.
input_folder = r'C:/Users/gades/Desktop/Thesis/datasets/terra_var/FFDI_finaldf90'  # Update to your folder path

# Get a sorted list of all TIFF files in the folder.
tif_files = sorted(glob.glob(os.path.join(input_folder, '*.tif')))

# Define the desired date range: from September 2013 to August 2023 (monthly).
desired_dates = pd.date_range(start='2013-09-01', end='2023-08-01', freq='MS')
n_desired = len(desired_dates)  # e.g., 120

# Filter the TIFF files to include only those corresponding to the desired period.
if len(tif_files) < n_desired:
    raise ValueError("Not enough TIFF files to cover the desired date range.")
tif_files = tif_files[:n_desired]
n_time = len(tif_files)
print(f"Using {n_time} files from {desired_dates[0].date()} to {desired_dates[-1].date()}.")

# Read the first file to get metadata (dimensions, etc.)
with rasterio.open(tif_files[0]) as src:
    height = src.height
    width = src.width
    profile = src.profile

print(f"Raster shape: ({height}, {width}).")

# Create a data cube to hold the time series data: (n_time, height, width)
data_cube = np.empty((n_time, height, width), dtype=np.float32)
for i, file in enumerate(tif_files):
    with rasterio.open(file) as src:
        data_cube[i, :, :] = src.read(1)

# Reshape the data cube so that each pixel's time series is a row.
n_pixels = height * width
pixel_timeseries_full = data_cube.reshape((n_time, n_pixels)).T  # Shape: (n_pixels, n_time)

# Create a mask for invalid (NaN) pixels.
invalid_mask = np.any(np.isnan(pixel_timeseries_full), axis=1)
valid_idx = ~invalid_mask  # Boolean index array for valid pixels

# Select only valid pixels for clustering.
pixel_timeseries = pixel_timeseries_full[valid_idx]

print(f"Number of valid pixels used for clustering: {pixel_timeseries.shape[0]}")

# ------------------------------
# K-Means Clustering on the Temporal Data
# ------------------------------
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
# Perform clustering on each valid pixel’s time series.
# The features are the temporal FFDI values.
cluster_labels_valid = kmeans.fit_predict(pixel_timeseries) + 1  # Shift labels to 1,2,3

# Create a full cluster map of size (n_pixels,) with NaN for invalid pixels.
cluster_map_flat = np.full((n_pixels,), np.nan)
# Fill in the valid pixel locations with cluster labels.
cluster_map_flat[valid_idx] = cluster_labels_valid
# Reshape back to the raster shape.
cluster_map = cluster_map_flat.reshape((height, width))

# ------------------------------
# Plot the Spatial Cluster Map (without x and y axis numbers)
# ------------------------------
plt.figure(figsize=(10, 8))
# Use a discrete colormap for clusters.
cmap = plt.get_cmap('viridis', n_clusters)
img = plt.imshow(cluster_map, cmap=cmap)
plt.title('Spatial Cluster Map of FFDI Pixels')

# Create a colorbar with cluster labels.
cbar = plt.colorbar(img, ticks=[1, 2, 3])
cbar.ax.set_yticklabels(['Cluster 1', 'Cluster 2', 'Cluster 3'])

# Remove x and y axis tick labels.
plt.xticks([])
plt.yticks([])

plt.tight_layout()

# Save the cluster map as an image if desired.
output_cluster_map = r'C:/Users/gades/Desktop/Thesis/outputs/ffdi_cluster_map.png'
os.makedirs(os.path.dirname(output_cluster_map), exist_ok=True)
plt.savefig(output_cluster_map, dpi=300)
print(f"Cluster map saved to: {output_cluster_map}")

plt.show()

## SECTION 6: Pixelwise Mann–Kendall Trend Mapping


Conducts Mann–Kendall trend test per pixel and saves a GeoTIFF indicating where significant upward or downward
trends occur.


In [ ]:
import os
import glob
import re
import numpy as np
import pandas as pd
import rasterio
from rasterio.windows import from_bounds
from pymannkendall import original_test  # Mann–Kendall test

# --- Configuration ---
folder_path   = r'C:/Users/gades/Desktop/Thesis/datasets/terra_var/FFDI_finaldf90'
output_trend  = r'C:/Users/gades/Desktop/Thesis/results/ffdi_mk_trend_map1123.tif'
date_pattern  = re.compile(r'(\d{4}-\d{2})')  # YYYY-MM in filename
NODATA_CODE   = 255                          # code for truly missing pixels

def extract_date(fp):
    m = date_pattern.search(os.path.basename(fp))
    return pd.to_datetime(m.group(1) + "-01") if m else pd.NaT

# --- Gather and sort rasters ---
tifs = sorted(glob.glob(os.path.join(folder_path, "*.tif")), key=extract_date)
if not tifs:
    raise RuntimeError("No TIFFs found")

# --- Read reference metadata ---
with rasterio.open(tifs[0]) as src0:
    meta    = src0.meta.copy()
    bounds  = src0.bounds
    height  = src0.height
    width   = src0.width

# --- Stack into (time, row, col) array ---
stack = np.empty((len(tifs), height, width), dtype=np.float32)
for i, fp in enumerate(tifs):
    with rasterio.open(fp) as src:
        window = from_bounds(*bounds, transform=src.transform)
        arr = src.read(
            1,
            window=window,
            out_shape=(height, width),
            resampling=rasterio.enums.Resampling.bilinear
        )
    stack[i] = arr

# --- Prepare trend_map with codes:
#    0 = no trend (p>=0.05)
#    1 = upward trend (p<0.05)
#    2 = downward trend (p<0.05)
trend_map = np.zeros((height, width), dtype=np.uint8)

# --- Mask of pixels with at least one valid observation ---
valid_mask = ~np.all(np.isnan(stack), axis=0)

# --- Per-pixel Mann–Kendall test ---
for i in range(height):
    for j in range(width):
        if not valid_mask[i, j]:
            continue
        ts = stack[:, i, j]
        if np.isnan(ts).any():
            continue
        res = original_test(ts)
        if res.p < 0.05:
            trend_map[i, j] = 1 if res.trend == "increasing" else 2

# --- Apply nodata code for pixels that never had data ---
trend_map_out = trend_map.copy()
trend_map_out[~valid_mask] = NODATA_CODE

# --- Write out as GeoTIFF with nodata=255 ---
meta.update({
    "driver":   "GTiff",
    "dtype":    rasterio.uint8,
    "count":    1,
    "nodata":   NODATA_CODE,
    "compress": "lzw"
})
os.makedirs(os.path.dirname(output_trend), exist_ok=True)
with rasterio.open(output_trend, "w", **meta) as dst:
    dst.write(trend_map_out, 1)

print(f"✅ Trend map saved to: {output_trend}")
# Codes: 0 = no trend, 1 = upward, 2 = downward, 255 = nodata

## SECTION 7: Harmonic Seasonal Decomposition of FFDI


Fits a seasonal harmonic model to 90th percentile FFDI values over time. Reveals dominant seasonal cycles
with amplitude and phase estimates.


In [ ]:
import os
import glob
import re
import numpy as np
import pandas as pd
import rasterio
from rasterio.windows import from_bounds
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
from scipy.signal import find_peaks

# ─── CONFIG ──────────────────────────────────────────────────────────────────
folder        = r"C:\Users\gades\Desktop\Thesis\datasets\terra_var\FFDI_finaldf90"
out_png       = r"C:\Users\gades\Desktop\Thesis\results\ffdi_harmonic_seasonality110.png"
date_pattern  = re.compile(r"(\d{4}-\d{2})")
fire_year_T   = 12    # months in a fire-year cycle
n_harmonics   = 2     # how many harmonics to fit

def extract_date(fp):
    m = date_pattern.search(os.path.basename(fp))
    return pd.to_datetime(m.group(1) + "-01") if m else pd.NaT

# ─── 1) LOAD & SORT TIFFs ────────────────────────────────────────────────────
files = sorted(glob.glob(os.path.join(folder, "*.tif")), key=extract_date)
dates = [extract_date(f) for f in files]
if not files:
    raise RuntimeError("No TIFFs found")

# ─── 2) READ META ─────────────────────────────────────────────────────────────
with rasterio.open(files[0]) as src0:
    bounds = src0.bounds
    h, w   = src0.height, src0.width

# ─── 3) STACK INTO (time, row, col) ──────────────────────────────────────────
stack = np.empty((len(files), h, w), dtype=np.float32)
for i, fp in enumerate(files):
    with rasterio.open(fp) as src:
        window = from_bounds(*bounds, transform=src.transform)
        arr = src.read(
            1, window=window,
            out_shape=(h, w),
            resampling=rasterio.enums.Resampling.bilinear
        )
    stack[i] = arr

# ─── 4) COMPUTE P90 + ENVELOPE ────────────────────────────────────────────────
p90  = np.nanpercentile(stack, 90, axis=(1,2))
minv = np.nanmin(stack, axis=(1,2))
maxv = np.nanmax(stack, axis=(1,2))

# ─── 5) FIRE-YEAR INDEX t ─────────────────────────────────────────────────────
first_sept = min(d for d in dates if d.month == 9)
t = np.array([(d.year - first_sept.year)*12 + (d.month - 9) for d in dates], dtype=float)

# ─── 6) HARMONIC MODEL & RESIDUALS ───────────────────────────────────────────
def harmonic_model(coefs, t, T, n):
    y = np.full_like(t, coefs[0], dtype=float)
    for k in range(1, n+1):
        ak = coefs[2*k-1]; bk = coefs[2*k]
        y += ak * np.cos(2*np.pi*k*t/T) + bk * np.sin(2*np.pi*k*t/T)
    return y

def residuals(coefs, t, T, n, obs):
    return harmonic_model(coefs, t, T, n) - obs

# ─── 7) FIT VIA LEAST-SQUARES ───────────────────────────────────────────────
init = np.zeros(1 + 2*n_harmonics)
init[0] = np.nanmean(p90)
res = least_squares(residuals, init, args=(t, fire_year_T, n_harmonics, p90))
coef = res.x

# ─── 8) EXTRACT AMPLITUDE & PHASE ────────────────────────────────────────────
mag, phase = {}, {}
for k in range(1, n_harmonics+1):
    ak = coef[2*k-1]; bk = coef[2*k]
    mag[k]   = np.hypot(ak, bk)
    phase[k] = np.degrees(np.arctan2(-bk, ak))

# ─── 9) PRINT RESULTS ─────────────────────────────────────────────────────────
print("Harmonic regression results:")
print(f" a₀ (mean) = {coef[0]:.2f}")
for k in range(1, n_harmonics+1):
    print(f" k={k}: amplitude = {mag[k]:.2f}, phase = {phase[k]:.1f}°")

# ─── 10) COMPUTE SERIES ───────────────────────────────────────────────────────
fitted = harmonic_model(coef, t, fire_year_T, n_harmonics)
components = {}
for k in range(1, n_harmonics+1):
    comp_coefs = np.zeros_like(coef)
    comp_coefs[2*k-1] = coef[2*k-1]
    comp_coefs[2*k]   = coef[2*k]
    comp_coefs[0]     = 0
    components[k] = harmonic_model(comp_coefs, t, fire_year_T, n_harmonics)

# ─── 11) PLOT & SAVE ─────────────────────────────────────────────────────────
plt.figure(figsize=(13, 5))
plt.fill_between(dates, minv, maxv, color='grey', alpha=0.5, label="Range of all pixels")
plt.plot(dates, p90, 'k-o', label="Observed Max FFDI")
plt.plot(dates, fitted, 'r-', linewidth=2, label="Fitted seasonal pattern")

linestyles = {1: ':', 2: '--'}
for k, comp in components.items():
    plt.plot(dates, comp, color=f'C{k}', linestyle=linestyles[k],
             label=f"Harmonic {k} (A={mag[k]:.1f}, φ={phase[k]:.0f}°)")

plt.axhline(y=35, color='crimson', linestyle='--', linewidth=1.5, label='FFDI Threshold = 35')

peak_idxs, _ = find_peaks(fitted, distance=2)
print("\n🔥 Peak months based on harmonic fit:")
for idx in peak_idxs:
    peak_date = dates[idx]
    print(f" → {peak_date.strftime('%Y-%m')} (FFDI ≈ {fitted[idx]:.2f})")
    start = peak_date - pd.DateOffset(months=1)
    end   = peak_date + pd.DateOffset(months=1)
    plt.axvspan(start, end, color='salmon', alpha=0.3)
    plt.text(peak_date, fitted[idx] + 1, peak_date.strftime('%b'),
             color='darkred', ha='center', fontsize=8, fontweight='bold')

# Create fire-year monthly date range for x-axis labels starting from first_sept
fire_year_dates = pd.date_range(start=first_sept, periods=len(dates), freq='MS')

# Format labels: full YYYY-MM for Septembers, month only otherwise
month_fmt = [d.strftime("%Y-%m") if d.month == 9 else d.strftime("%m") for d in fire_year_dates]

plt.xticks(fire_year_dates, month_fmt, rotation=90)
plt.xlim(fire_year_dates[0], fire_year_dates[-1])

plt.xlabel("Fire Year Month")
plt.ylabel("FFDI")
plt.grid(linestyle=':', alpha=0.6)
plt.legend(ncol=1, fontsize=9)
plt.tight_layout()

os.makedirs(os.path.dirname(out_png), exist_ok=True)
plt.savefig(out_png, dpi=300)
plt.show()

print(f"\n✅ Seasonal plot saved to: {out_png}")

## SECTION 8: December–January–February Block Analysis by Cluster


Focuses on the core fire season (Dec–Feb), aggregates FFDI over 3-month blocks, and checks for cluster-wise
seasonal trends.


In [ ]:
import os
import glob
import re
import numpy as np
import pandas as pd
import rasterio
from rasterio.windows import from_bounds
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from pymannkendall import original_test, sens_slope

# ─── CONFIG ──────────────────────────────────────────────────────────────────
ffdi_folder  = r"C:\Users\gades\Desktop\Thesis\datasets\terra_var\FFDI_finaldf90"
n_clusters   = 3
start_year   = 2013
end_year     = 2023    # last block = 2022-12→2023-02
out_png      = r"C:\Users\gades\Desktop\Thesis\results\decjanfeb_full_series_fixed.png"
date_pat     = re.compile(r"(\d{4}-\d{2})")

def extract_date(fp):
    m = date_pat.search(os.path.basename(fp))
    return pd.to_datetime(m.group(1) + "-01") if m else pd.NaT

# 1) Load & sort monthly TIFFs
files = sorted(glob.glob(os.path.join(ffdi_folder, "*.tif")), key=extract_date)
dates = [extract_date(f) for f in files]
if not files:
    raise RuntimeError("No TIFFs found")

# 2) Read metadata & stack
with rasterio.open(files[0]) as src0:
    bounds = src0.bounds
    h, w   = src0.height, src0.width

stack = np.empty((len(files), h, w), dtype=np.float32)
for i, fp in enumerate(files):
    with rasterio.open(fp) as src:
        win = from_bounds(*bounds, transform=src.transform)
        stack[i] = src.read(
            1, window=win,
            out_shape=(h, w),
            resampling=rasterio.enums.Resampling.bilinear
        )

# 3) Cluster by long‐term mean
pix = stack.reshape(len(files), -1).T
pix = SimpleImputer(strategy="mean").fit_transform(pix)
pix_mean   = pix.mean(axis=1).reshape(h, w)
labels     = KMeans(n_clusters=n_clusters, random_state=0)\
                 .fit_predict(pix_mean.flatten().reshape(-1,1))
cluster_map = labels.reshape(h, w)

# 4) Aggregate Dec→Feb blocks (3 months)
records = []
for k in range(n_clusters):
    mask = (cluster_map == k)
    for yr in range(start_year, end_year+1):
        block_dates = [
            pd.Timestamp(f"{yr}-12-01"),
            pd.Timestamp(f"{yr+1}-01-01"),
            pd.Timestamp(f"{yr+1}-02-01"),
        ]
        # need all three months present
        if not all(d in dates for d in block_dates):
            continue
        idxs = [i for i,d in enumerate(dates) if d in block_dates]
        vals = stack[idxs][:, mask]
        records.append({
            "cluster":   k+1,
            "plot_date": block_dates[0],           # December of each block
            "label":     f"{yr}-12→{yr+1}-02",
            "mean_ffdi": np.nanmean(vals)
        })

df = pd.DataFrame(records)
df.sort_values(["plot_date","cluster"], inplace=True)

# 5) Pivot so clusters are columns
pivot = df.pivot(index="plot_date", columns="cluster", values="mean_ffdi")

# 6) Reindex to ensure full series 2013–2022 Dec starts
full_index = pd.date_range(
    start=f"{start_year}-12-01",
    periods=(end_year-start_year+1),
    freq=pd.DateOffset(years=1)
)
pivot = pivot.reindex(full_index)

# 7) Plot with trends
plt.figure(figsize=(12,6))
colors = {1:"C0", 2:"C1", 3:"C2"}
trend_info = {}

for cl in pivot.columns:
    ts = pivot[cl]
    y  = ts.values
    # drop NaN for trend calc
    notnan = ~np.isnan(y)
    y_nn = y[notnan]
    x_nn = np.arange(len(y_nn))

    mk = original_test(y_nn)
    ss = sens_slope(y_nn)
    trend_info[cl] = (mk.trend, mk.p, ss.slope)

    # series
    plt.plot(ts.index, y,
             color=colors[cl], marker='o', linestyle='-',
             label=f"Cluster {cl} Mean")
    # trend line
    plt.plot(ts.index[notnan], y_nn[0] + ss.slope*x_nn,
             color=colors[cl], linestyle='--',
             label=f"C{cl} Trend (s={ss.slope:.2f}, p={mk.p:.2f})")

# X-axis ticks at each block
xticks  = full_index
xlabels = [f"{d.year}-12→{d.year+1}-02" for d in full_index]
plt.xticks(xticks, xlabels, rotation=45)
plt.xlim(xticks[0], xticks[-1])

plt.xlabel("Dec - Feb")
plt.ylabel("Mean FFDI")
plt.grid(linestyle=':', alpha=0.5)
plt.legend(fontsize=8, ncol=1)
plt.tight_layout()

os.makedirs(os.path.dirname(out_png), exist_ok=True)
plt.savefig(out_png, dpi=300)
plt.show()

# 8) Summary
print("Trend analysis results:")
for cl, (trend, p, slope) in trend_info.items():
    print(f" Cluster {cl}: trend={trend}, p={p:.3f}, slope={slope:.4f}")
print(f"\n✅ Plot saved to: {out_png}")

## SECTION 9: High- Risk Area Calculation


In [ ]:
import os
import math
import rasterio
import numpy as np
import pandas as pd
from glob import glob

# --- CONFIG ---
ffdi_folder = r"C:\Users\gades\Desktop\Thesis\datasets\terra_var\FFDI_finaldf90"
ffdi_threshold > 12
output_csv = os.path.join(ffdi_folder, "high_risk_area_by_year.csv")

# --- FUNCTION TO CONVERT DEGREE PIXEL SIZE TO KM² (based on NSW latitude) ---
def degrees_to_km2(pixel_width_deg, pixel_height_deg, lat=-33.5):
    """
    Estimate km² per pixel for given latitude (defaults to NSW ~ -33.5°)
    """
    km_per_deg_lat = 111.32
    km_per_deg_lon = 111.32 * math.cos(math.radians(lat))
    return (pixel_width_deg * km_per_deg_lon) * (pixel_height_deg * km_per_deg_lat)

# --- EXTRACT YEAR FROM FILENAME ---
def year_from_filename(fname):
    base = os.path.basename(fname)
    return base.split("_")[1][:4]

# --- MAIN ---
files = sorted(glob(os.path.join(ffdi_folder, "ffdi_*.tif")))
yearly_files = {}
for f in files:
    year = year_from_filename(f)
    yearly_files.setdefault(year, []).append(f)

results = []

for year, rasters in yearly_files.items():
    high_risk_mask = None
    total_pixels = None
    pixel_width = None
    pixel_height = None

    for raster_path in rasters:
        with rasterio.open(raster_path) as src:
            data = src.read(1)
            mask = (data > ffdi_threshold)

            if high_risk_mask is None:
                high_risk_mask = mask
                total_pixels = np.isfinite(data).sum()
                transform = src.transform
                pixel_width = transform[0]
                pixel_height = abs(transform[4])
            else:
                high_risk_mask |= mask  # Logical OR across months

    high_risk_pixel_count = np.sum(high_risk_mask)

    # Estimate area in km² based on NSW latitude
    pixel_area_km2 = degrees_to_km2(pixel_width, pixel_height, lat=-33.5)
    area_km2 = high_risk_pixel_count * pixel_area_km2

    results.append({
        "Year": int(year),
        "HighRiskPixels": int(high_risk_pixel_count),
        "TotalPixels": int(total_pixels),
        "HighRiskArea_km2": round(area_km2, 2)
    })

# --- EXPORT ---
df = pd.DataFrame(results).sort_values("Year")
df.to_csv(output_csv, index=False)

# --- PRINT ---
print("\nHigh-Risk Area Summary by Year (NSW, km²-based):\n")
print(df.to_string(index=False))


## SECTION 10: Year-wise Line Plot of Total High-Risk Area


Uses a CSV file containing yearly summaries of high fire danger area to plot interannual variation in total
area under fire risk.


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# ─── UPDATE TO YOUR CSV PATH ─────────────────────────────────────────────────
highrisk_csv = r"C:\Users\gades\Desktop\Thesis\results\fireyear_unique_highrisk.csv"

if not os.path.exists(highrisk_csv):
    raise FileNotFoundError(f"CSV not found: {highrisk_csv}")

# ─── LOAD DATA ───────────────────────────────────────────────────────────────
df = pd.read_csv(highrisk_csv)

# Extract the fire‐year start as an integer (e.g., "2013-09→2014-08" → 2013)
df["start_year"] = df["fire_year"].str.slice(0,4).astype(int)

# ─── PLOT LINE CHART ─────────────────────────────────────────────────────────
plt.figure(figsize=(10, 6))
plt.plot(df["start_year"], df["area_km2"], marker="D", linestyle="-", color="C0", linewidth=2)


plt.xlabel("Fire Year", fontsize=12)
plt.ylabel("High-Risk Extent (km²)", fontsize=12)
plt.xticks(df["start_year"], rotation=0)
plt.ylim(0, df["area_km2"].max() * 1.1)
plt.grid(True, linestyle=":", linewidth=0.5, alpha=0.7)
plt.tight_layout()

# ─── SAVE & SHOW ────────────────────────────────────────────────────────────
output_line = r"C:\Users\gades\Desktop\Thesis\results\line_fireyear_highrisk_area1.png"
os.makedirs(os.path.dirname(output_line), exist_ok=True)
plt.savefig(output_line, dpi=300)
plt.show()

print(f"✅ Line chart saved to: {output_line}")

In [ ]:
pip install pymannkendall

## SECTION 11: Multi-Map Visualization of FFDI Trends (DJF and Full Series)


This section creates a six-panel visualization summarizing trend direction, slope, and significance (p-value)
for both the full series and the December–January–February (DJF) season. Each map shows spatial patterns
helpful for understanding fire danger trends over time.

The panels include:
- Trend direction: increasing, decreasing, or no trend
- Sen's slope (rate of change)
- p-value (significance)

A scale bar and north arrow are included for spatial context.


In [ ]:
import os
import glob
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio
from rasterio.windows import from_bounds
from matplotlib.colors import ListedColormap
from matplotlib_scalebar.scalebar import ScaleBar
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pymannkendall import original_test, sens_slope

# ─── CONFIG ───────────────────────────────────────────────────────────────────
input_folder   = r"C:/Users/gades/Desktop/Thesis/datasets/terra_var/FFDI_finaldf90"
pattern        = re.compile(r"ffdi_(\d{4}-\d{2})")
nodata_val     = -9999
out_image      = r"C:/Users/gades/Desktop/Thesis/results/ffdi_trend_maps.png"

# ─── FUNCTION TO EXTRACT DATE FROM FILENAME ───────────────────────────────────
def extract_date(path):
    m = pattern.search(os.path.basename(path))
    return pd.to_datetime(m.group(1) + "-01") if m else pd.NaT

# ─── LOAD FILES ───────────────────────────────────────────────────────────────
tif_files = sorted(glob.glob(os.path.join(input_folder, "*.tif")), key=extract_date)
dates     = [extract_date(f) for f in tif_files]
if not tif_files:
    raise RuntimeError("No FFDI TIFF files found")

# ─── READ REFERENCE ───────────────────────────────────────────────────────────
with rasterio.open(tif_files[0]) as src0:
    height, width = src0.height, src0.width
    transform     = src0.transform
    crs           = src0.crs
    bounds        = src0.bounds

# ─── STACK FFDI ────────────────────────────────────────────────────────────────
full_stack  = np.empty((len(tif_files), height, width), dtype=np.float32)
for i, f in enumerate(tif_files):
    with rasterio.open(f) as src:
        win = from_bounds(*bounds, transform=src.transform)
        full_stack[i] = src.read(1, window=win, out_shape=(height, width))

full_dates = np.array(dates)

# ─── CREATE SEASONAL (DJF) STACK ──────────────────────────────────────────────
is_djf = [(d.month in [12, 1, 2]) for d in full_dates]
djf_stack = full_stack[is_djf]
djf_dates = full_dates[is_djf]

# ─── STATISTICAL ANALYSIS FUNCTION ────────────────────────────────────────────
def compute_stats(stack, dates):
    trend_map  = np.full((height, width), 0, dtype=np.int8)     # -1, 0, 1
    slope_map  = np.full((height, width), np.nan, dtype=np.float32)
    pval_map   = np.full((height, width), np.nan, dtype=np.float32)
    valid_mask = ~np.all(np.isnan(stack), axis=0)

    for i in range(height):
        for j in range(width):
            if not valid_mask[i, j]:
                continue
            ts = stack[:, i, j]
            if np.isnan(ts).any():
                continue
            mk = original_test(ts)
            ss = sens_slope(ts)
            trend_map[i, j] = {"increasing": 1, "decreasing": -1}.get(mk.trend, 0)
            pval_map[i, j]  = mk.p
            slope_map[i, j] = ss.slope
    return trend_map, slope_map, pval_map

# ─── RUN STATS ────────────────────────────────────────────────────────────────
trend_full, slope_full, pval_full = compute_stats(full_stack, full_dates)
trend_djf,  slope_djf,  pval_djf  = compute_stats(djf_stack,  djf_dates)

# ─── PLOTTING SETUP ───────────────────────────────────────────────────────────
def plot_map(ax, data, title, cmap, vmin=None, vmax=None, is_discrete=False, cbar_label="", is_trend=False):
    im = ax.imshow(data, cmap=cmap, vmin=vmin, vmax=vmax, extent=[*bounds[:2], *bounds[2:]], origin='upper')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="3%", pad=0.05)
    cbar = plt.colorbar(im, cax=cax)
    cbar.ax.set_ylabel(cbar_label)
    if is_trend:
        cbar.set_ticks([-1, 0, 1])
        cbar.set_ticklabels(['Decreasing', 'No trend', 'Increasing'])
    ax.set_title(title, fontsize=10)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")

def add_scalebar_and_north(ax):
    sb = ScaleBar(dx=1, units='m', dimension='si-length', location='lower left')
    ax.add_artist(sb)
    ax.annotate('N', xy=(0.95, 0.1), xycoords='axes fraction', ha='center', va='center',
                fontsize=12, fontweight='bold', arrowprops=dict(facecolor='black', width=5, headwidth=15))

# ─── PLOT ALL 6 MAPS ──────────────────────────────────────────────────────────
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
plt.subplots_adjust(hspace=0.3, wspace=0.3)

# Colormaps
trend_cmap = ListedColormap(['red', 'lightgray', 'green'])
slope_cmap = 'coolwarm'
pval_cmap  = 'viridis_r'

# Full series
plot_map(axes[0, 0], trend_full, "Trend (Full Series)", trend_cmap, vmin=-1, vmax=1, is_discrete=True, is_trend=True)
plot_map(axes[0, 1], slope_full, "Sen's Slope (Full Series)", slope_cmap, cbar_label="Slope")
plot_map(axes[0, 2], pval_full, "p-value (Full Series)", pval_cmap, vmin=0, vmax=1, cbar_label="p")

# DJF
plot_map(axes[1, 0], trend_djf, "Trend (DJF Only)", trend_cmap, vmin=-1, vmax=1, is_discrete=True, is_trend=True)
plot_map(axes[1, 1], slope_djf, "Sen's Slope (DJF Only)", slope_cmap, cbar_label="Slope")
plot_map(axes[1, 2], pval_djf, "p-value (DJF Only)", pval_cmap, vmin=0, vmax=1, cbar_label="p")

# Add north arrow and scalebar to one map
add_scalebar_and_north(axes[1, 0])

# Save and show
plt.suptitle("FFDI Trend Analysis Maps", fontsize=14)
plt.tight_layout(rect=[0, 0, 1, 0.96])
os.makedirs(os.path.dirname(out_image), exist_ok=True)
plt.savefig(out_image, dpi=300)
plt.show()

print(f"\n✅ 6-map summary image saved to: {out_image}")
